In [ ]:
!pip install dash dash_bootstrap_components binance

In [ ]:
import pandas as pd
import numpy as np
import pandas_ta as ta
import requests
import websocket
import json
import threading
import queue
import time
from datetime import datetime, timedelta
import pytz
import logging
import yfinance as yf
from backtesting import Strategy, Backtest
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.nonparametric.kernel_regression import KernelReg

# Configuration
BINANCE_BASE_URL = "https://api.binance.com"
BINANCE_WS_URL = "wss://stream.binance.com:9443/ws"
SYMBOL = "BTCUSDT"
INTERVAL = "5m"
LOOKBACK_DAYS = 1
MAX_RETRIES = 3
RETRY_DELAY = 5

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Initialize global variables
btc_df = pd.DataFrame()
sp500_df = pd.DataFrame()
data_queue = queue.Queue()

def fetch_binance_klines(symbol, interval, start_time, end_time):
    endpoint = f"{BINANCE_BASE_URL}/api/v3/klines"
    params = {
        "symbol": symbol,
        "interval": interval,
        "startTime": int(start_time.timestamp() * 1000),
        "endTime": int(end_time.timestamp() * 1000),
        "limit": 1000
    }

    for attempt in range(MAX_RETRIES):
        try:
            response = requests.get(endpoint, params=params)
            response.raise_for_status()
            data = response.json()

            df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True)
            df.set_index('timestamp', inplace=True)
            df = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
            df.index = df.index.tz_convert('Europe/London')

            logger.info(f"Successfully fetched {len(df)} Binance candles")
            return df

        except requests.RequestException as e:
            logger.warning(f"Attempt {attempt + 1} failed: {e}")
            if attempt < MAX_RETRIES - 1:
                time.sleep(RETRY_DELAY * (2 ** attempt))  # Exponential backoff
            else:
                logger.error(f"Failed to fetch Binance data after {MAX_RETRIES} attempts")
                return pd.DataFrame()

def fetch_sp500_data(start_date, end_date, interval='5m'):
    try:
        sp500 = yf.Ticker("^GSPC")
        df = sp500.history(start=start_date, end=end_date, interval=interval)
        df.index = df.index.tz_convert('Europe/London')
        logger.info(f"Successfully fetched {len(df)} S&P500 candles")
        return df
    except Exception as e:
        logger.error(f"Error fetching S&P500 data: {e}")
        return pd.DataFrame()

def calculate_indicators(df, ema_slow=50, ema_fast=20):
    df['EMA_slow'] = ta.ema(df['close'], length=ema_slow)
    df['EMA_fast'] = ta.ema(df['close'], length=ema_fast)
    df['ATR'] = ta.atr(df['high'], df['low'], df['close'], length=14)
    return df

def calculate_nadaraya_watson(df, bandwidth=0.1):
    X = np.arange(len(df)).reshape(-1, 1)
    y = df['close'].values
    model = KernelReg(endog=y, exog=X, var_type='c', bw=[bandwidth])
    df['NW_Fitted'], _ = model.fit(X)
    residuals = df['close'] - df['NW_Fitted']
    std_dev = np.std(residuals)
    df['Upper_Envelope'] = df['NW_Fitted'] + 2 * std_dev
    df['Lower_Envelope'] = df['NW_Fitted'] - 2 * std_dev
    return df

def calculate_sp500_gamma(df):
    df['Gamma'] = (df['Close'] - df['Open']) / df['Open'] * 100
    return df

def generate_signals(btc_df, sp500_df):
    common_index = btc_df.index.intersection(sp500_df.index)
    btc_df = btc_df.loc[common_index]
    sp500_df = sp500_df.loc[common_index]

    btc_df['SP500_Signal'] = np.where(sp500_df['Close'] > sp500_df['EMA_fast'], 1, 0)
    btc_df['Gamma_Signal'] = np.where(sp500_df['Gamma'] > 0, 1, 0)
    btc_df['BTC_Signal'] = np.where((btc_df['close'] > btc_df['Upper_Envelope']) & (btc_df['EMA_fast'] > btc_df['EMA_slow']), 1,
                                    np.where((btc_df['close'] < btc_df['Lower_Envelope']) & (btc_df['EMA_fast'] < btc_df['EMA_slow']), -1, 0))
    btc_df['Total_Signal'] = btc_df['SP500_Signal'] + btc_df['Gamma_Signal'] + btc_df['BTC_Signal']

    return btc_df

def on_message(ws, message):
    data = json.loads(message)

    if data['e'] == 'kline':
        kline = data['k']

        if kline['x']:  # If candle closed
            timestamp = pd.to_datetime(kline['T'], unit='ms', utc=True).tz_convert('Europe/London')
            new_data = {
                'open': float(kline['o']),
                'high': float(kline['h']),
                'low': float(kline['l']),
                'close': float(kline['c']),
                'volume': float(kline['v'])
            }
            data_queue.put((timestamp, new_data))

def on_error(ws, error):
    logger.error(f"WebSocket error: {error}")

def on_close(ws):
    logger.info("WebSocket connection closed")

def on_open(ws):
    logger.info("WebSocket connection opened")
    subscribe_message = {
        "method": "SUBSCRIBE",
        "params": [f"{SYMBOL.lower()}@kline_{INTERVAL}"],
        "id": 1
    }
    ws.send(json.dumps(subscribe_message))

def websocket_thread():
    while True:
        try:
            ws = websocket.WebSocketApp(BINANCE_WS_URL,
                                        on_message=on_message,
                                        on_error=on_error,
                                        on_close=on_close,
                                        on_open=on_open)
            ws.run_forever()
        except Exception as e:
            logger.error(f"WebSocket error: {e}. Reconnecting...")
            time.sleep(5)

class RefinedIntraDayStrategy(Strategy):
    mysize = 1.0
    slcoef = 1.5
    TPSLRatio = 2.0

    def init(self):
        super().init()
        self.signal = self.I(lambda: self.data.Total_Signal)

    def next(self):
        super().next()

        if self.signal > 1 and not self.position:
            sl = self.data.Close[-1] - self.data.ATR[-1] * self.slcoef
            tp = self.data.Close[-1] + self.data.ATR[-1] * self.slcoef * self.TPSLRatio
            self.buy(sl=sl, tp=tp, size=self.mysize)
        elif self.signal < -1 and not self.position:
            sl = self.data.Close[-1] + self.data.ATR[-1] * self.slcoef
            tp = self.data.Close[-1] - self.data.ATR[-1] * self.slcoef * self.TPSLRatio
            self.sell(sl=sl, tp=tp, size=self.mysize)

def plot_strategy(btc_df, sp500_df, trades):
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                        subplot_titles=('BTC/USD', 'S&P 500', 'S&P 500 Gamma Exposure'))

    fig.add_trace(go.Candlestick(x=btc_df.index, open=btc_df['open'], high=btc_df['high'],
                                 low=btc_df['low'], close=btc_df['close'], name='BTC/USD'), row=1, col=1)
    fig.add_trace(go.Scatter(x=btc_df.index, y=btc_df['Upper_Envelope'], name='Upper Envelope',
                             line=dict(color='rgba(255,0,0,0.3)', width=1, dash='dot')), row=1, col=1)
    fig.add_trace(go.Scatter(x=btc_df.index, y=btc_df['Lower_Envelope'], name='Lower Envelope',
                             line=dict(color='rgba(0,255,0,0.3)', width=1, dash='dot')), row=1, col=1)

    fig.add_trace(go.Candlestick(x=sp500_df.index, open=sp500_df['Open'], high=sp500_df['High'],
                                 low=sp500_df['Low'], close=sp500_df['Close'], name='S&P 500'), row=2, col=1)

    fig.add_trace(go.Scatter(x=sp500_df.index, y=sp500_df['Gamma'], name='S&P 500 Gamma',
                             line=dict(color='purple', width=1)), row=3, col=1)

    for trade in trades:
        fig.add_trace(go.Scatter(x=[trade.EntryTime], y=[trade.EntryPrice],
                                 mode='markers', marker=dict(symbol='triangle-up' if trade.Size > 0 else 'triangle-down',
                                                             size=10, color='green' if trade.Size > 0 else 'red'),
                                 name='Entry'), row=1, col=1)
        if trade.ExitTime:
            fig.add_trace(go.Scatter(x=[trade.ExitTime], y=[trade.ExitPrice],
                                     mode='markers', marker=dict(symbol='circle', size=10, color='blue'),
                                     name='Exit'), row=1, col=1)

    fig.update_layout(height=1200, title_text="BTC-S&P500 Intraday Strategy with Gamma Exposure")
    fig.show()

def main():
    global btc_df, sp500_df

    # Fetch initial historical data
    end_time = datetime.now(pytz.UTC)
    start_time = end_time - timedelta(days=LOOKBACK_DAYS)

    btc_df = fetch_binance_klines(SYMBOL, INTERVAL, start_time, end_time)
    sp500_df = fetch_sp500_data(start_time, end_time, interval=INTERVAL)

    if btc_df.empty or sp500_df.empty:
        logger.error("Unable to fetch initial data. Exiting.")
        return

    # Start WebSocket thread
    ws_thread = threading.Thread(target=websocket_thread)
    ws_thread.start()

    while True:
        try:
            # Process new data from WebSocket
            while not data_queue.empty():
                timestamp, new_data = data_queue.get()
                btc_df.loc[timestamp] = new_data

            # Keep only the last 24 hours of data
            btc_df = btc_df.last('24H')

            # Recalculate indicators and signals
            btc_df = calculate_indicators(btc_df)
            btc_df = calculate_nadaraya_watson(btc_df)
            sp500_df = calculate_sp500_gamma(sp500_df)
            btc_df = generate_signals(btc_df, sp500_df)

            # Run backtest
            bt = Backtest(btc_df, RefinedIntraDayStrategy, cash=100000, commission=.002)
            stats = bt.run()

            logger.info(f"Strategy performance: {stats['Return [%]']:.2f}% return")

            # Plot results every hour
            if datetime.now().minute == 0:
                plot_strategy(btc_df, sp500_df, stats._trades)

            time.sleep(60)  # Wait for 1 minute before next update

        except Exception as e:
            logger.error(f"Error in main loop: {e}")
            time.sleep(60)  # Wait for 1 minute before retrying

if __name__ == "__main__":
    main()

2024-09-13 21:16:34,989 - INFO - Successfully fetched 288 Binance candles
2024-09-13 21:16:35,767 - INFO - Successfully fetched 78 S&P500 candles
2024-09-13 21:16:36,085 - ERROR - Error in main loop: 'EMA_fast'


--- request header ---
2024-09-13 21:16:37,087 - DEBUG - --- request header ---
GET /ws HTTP/1.1
Upgrade: websocket
Host: stream.binance.com:9443
Origin: https://stream.binance.com:9443
Sec-WebSocket-Key: sjYMs6tvz2nXOidMk+vDJQ==
Sec-WebSocket-Version: 13
Connection: Upgrade


2024-09-13 21:16:37,158 - DEBUG - GET /ws HTTP/1.1
Upgrade: websocket
Host: stream.binance.com:9443
Origin: https://stream.binance.com:9443
Sec-WebSocket-Key: sjYMs6tvz2nXOidMk+vDJQ==
Sec-WebSocket-Version: 13
Connection: Upgrade


-----------------------
2024-09-13 21:16:37,162 - DEBUG - -----------------------
--- response header ---
2024-09-13 21:16:37,164 - DEBUG - --- response header ---
HTTP/1.1 101 Switching Protocols
2024-09-13 21:16:37,353 - DEBUG - HTTP/1.1 101 Switching Protocols
Date: Fri, 13 Sep 2024 21:16:37 GMT
2024-09-13 21:16:37,359 - DEBUG - Date: Fri, 13 Sep 2024 21:16:37 GMT
Connection: upgrade
2024-09-13 21:16:37,363 - DEBUG - Connection: upgrade
Upgrade: websocket
2024-09-13 21:16:37,364 - D

KeyboardInterrupt: 

++Rcv raw: b'\x81~\x01O{"e":"kline","E":1726262242712,"s":"BTCUSDT","k":{"t":1726262100000,"T":1726262399999,"s":"BTCUSDT","i":"5m","f":3818446414,"L":3818460530,"o":"60084.01000000","c":"60127.01000000","h":"60224.00000000","l":"60050.00000000","v":"103.77231000","n":14117,"x":false,"q":"6242484.24338310","V":"42.77493000","Q":"2573290.53338540","B":"0"}}'
2024-09-13 21:17:22,897 - DEBUG - ++Rcv raw: b'\x81~\x01O{"e":"kline","E":1726262242712,"s":"BTCUSDT","k":{"t":1726262100000,"T":1726262399999,"s":"BTCUSDT","i":"5m","f":3818446414,"L":3818460530,"o":"60084.01000000","c":"60127.01000000","h":"60224.00000000","l":"60050.00000000","v":"103.77231000","n":14117,"x":false,"q":"6242484.24338310","V":"42.77493000","Q":"2573290.53338540","B":"0"}}'
++Rcv decoded: fin=1 opcode=1 data=b'{"e":"kline","E":1726262242712,"s":"BTCUSDT","k":{"t":1726262100000,"T":1726262399999,"s":"BTCUSDT","i":"5m","f":3818446414,"L":3818460530,"o":"60084.01000000","c":"60127.01000000","h":"60224.00000000","l":"60